# This code is right now executed in google colab. The date as of completing this code is 14/02/2022. Right now, this code will run without any problems while setting the runtime on TPU. If you set the runtime on GPU, this code won't work right now. 

# Yes, it may happen in future that the problem prevalent with setting runtine as GPU, gets solved so if you are trying to execute this code after let's say a month or two, please first try setting up the runtime to GPU because on GPU, it gets trained faster and it may be the case that it gets executed without any problems. On TPU runtime, it gets trained slowly. 

# The working link to download the dataset has been provided inside the code. 

# Problem Statement:

##### Given images of the different cigarrette packets on different aisles across different grocery stores, you have to train an object detection neural network to perform object detection on cigarrette packets of different brands so that the number of cigarrette packets of different brands can be counted and hence their availability can be tracked across different grocery stores and therefore better decisions can be taken regarding the supply chain of different brands of cigarette packets so as to ensure their continuous availability to the customer.   

# Let's navigate inside the root directory of google drive. 

In [ ]:
cd /content/drive/MyDrive

#Let's create a directory to clone all the files and folders of Tensorflow 2.0 Object Detection API. 

In [ ]:
!mkdir tf2_od_api

# Let's now clone the repository of Tensorflow 2.0 Object Detection API files inside this directory. 

In [ ]:
cd /content/drive/MyDrive/tf2_od_api

In [ ]:
! git clone https://github.com/tensorflow/models.git

# Let's now follow the steps to install Tensorflow 2.0 Object Detection API. 

In [ ]:
! sudo apt-get install protobuf-compiler python-pil python-lxml python-tk
! pip install Cython contextlib2 pillow lxml

In [ ]:
import os
os.environ["PYTHONPATH"] += ':/content/models/research/:/content/models/research/slim/'

# Let's now navigate inside a specific directory named "research" inside the installtion directory of Tensorflow 2.0 Object Detection API, tensorflow

In [ ]:
cd /content/drive/MyDrive/tf2_od_api/models/research

In [ ]:
! protoc object_detection/protos/*.proto --python_out=.

# Let's again navigate back into the root directory of google drive. 

In [ ]:
cd /content/drive/MyDrive

# Let's clone the MS COCO API repository in the root directory of google drive. 

In [ ]:
! git clone https://github.com/cocodataset/cocoapi.git

# Let's navigate into the directory of MS COCO API. 

In [ ]:
cd cocoapi/PythonAPI

# Let's now start the installation of MS COCO API

In [ ]:
! make

In [ ]:
cp -r pycocotools /content/drive/MyDrive/tf2_od_api/models/research

# Let's navigate back into the "research" directory inside installation directory. 

In [ ]:
cd /content/drive/MyDrive/tf2_od_api/models/research

# Let's copy some files from this directory. 

In [ ]:
cp object_detection/packages/tf2/setup.py .

In [ ]:
! python -m pip install --use-feature=2020-resolver .

# Let's now check that whether the installtion of our Tensorflow 2 Object Detection API is successful or not. 

In [ ]:
! python object_detection/builders/model_builder_tf2_test.py

# The "OK" at the end signifies that the installation of our Tensorflow 2 Object Detection API has been successful. 

# As you can see above the installation of Tensorflow 2 Object Detection API has been succeeded. Now, we have to train our desired neural network model to detect objects in images of our dataset.  

# So firstly, please navigate to the installtion directory of the API.  

In [ ]:
cd /content/drive/MyDrive/tf2_od_api

# Let's first create a directory named "workspace" inside our navigated directory of installation. In this directory of "workspace", we will be saving the training configurations of different neural network archirectures on different datasets. 

In [ ]:
! mkdir workspace

# Now, inside this workspace directory, we will be creating the directories for training configurations on different datasets. A different dataset will be having different directory. Now, because we will be here performing object detection on shelf images, so let's create a directory named "shelf_objects_detection" inside this directory (workspace). 

In [ ]:
cd /content/drive/MyDrive/tf2_od_api/workspace

In [ ]:
! mkdir ciggs_images_detection

# Now, every such directory will consist of all the details of training configurations related to object detection for a specific dataset. 

# Let's create a directory inside "shelf_objects_detection", named "images" inside which we will be downloading our training as well as testing datasets. 

In [ ]:
cd /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection

In [ ]:
! mkdir images

# Now, let's download our dataset into this directory. 

In [ ]:
cd /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/images

In [ ]:
! wget https://storage.googleapis.com/open_source_datasets/ShelfImages.tar.gz

# Now, let's unzip the dataset into this directory. 

In [ ]:
! tar -xvf /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/images/ShelfImages.tar.gz

# Now create the following directories inside /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection

# annotations
# exported_models
# models
# pre_trained_models

# Now, please remember that if you are going to use Tensorflow 2 Object Detection API to detect objects in images of some other dataset then you have to create a new directory inside /content/drive/MyDrive/tensorflow_od_api/workspace:

# For Example, /content/drive/MyDrive/tf2_od_api/workspace/ex_dark

# and the rest of the directory struture will remain the same. 



In [ ]:
cd /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection

In [ ]:
! mkdir annotations
! mkdir exported_models
! mkdir models
! mkdir pre_trained_models

# Every single directory has some purpose which will be discussing about later on. So, right now we have following directories inside /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection 

# annotations
# exported_models
# images
# models
# pre_trained_models

# Let's now download the annotations of our Shelf Objects Images dataset in the annotations directory. 

In [ ]:
cd /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/annotations

In [ ]:
! git clone https://github.com/gulvarol/grocerydataset.git

In [ ]:
! python -m pip uninstall opencv-python-headless==4.5.5.62
! python -m pip install opencv-python-headless==4.1.2.30

In [ ]:
import os
import glob
import pandas as pd
import io
import tensorflow.compat.v1 as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Now, let's create our label map file. But the question arrises is what is label map file ? Label map file will consist of dictionaries for different classes of objects which can be detected in an image. The extension of this file is .pbtxt. Each dictionary will consist of two key value pairs, where first key value pair maps the 'id' variable to the class label of the object and the second key value pair maps the 'name' variable to the name of the object. A sample of a dictionary for an object in label map looks like this: 

# item {
#     id: 1
#     name: "Category 1 Object"
# }




# As there are 11 different objects in our shelf images dataset, therefore there will be 11 such dictionaries written in this label_map.pbtxt file. 

# This label_map.pbtxt file will be written inside the following directory: 

# /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/annotations

In [ ]:
cp /content/drive/MyDrive/label_map.pbtxt /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/annotations/

# We have created the label_map.pbtxt file and written it inside the above mentioned directory. Now, let's load this file and fetch the dictionary from the file. 

In [ ]:
label_map = label_map_util.load_labelmap("/content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/annotations/label_map.pbtxt")
label_map_dict = label_map_util.get_label_map_dict(label_map)

In [ ]:
label_map_dict_copy = dict()

for k,v in label_map_dict.items():
  label_map_dict_copy[v] = k

label_map_dict = label_map_dict_copy

In [ ]:
label_map_dict

# Let's navigate back to the root directory of google drive

In [ ]:
cd /content/drive/MyDrive

# Let's write some functions to create .tfrecord files to store our examples in the form of images as well as it's ground truth labels in the form of ground truth bounding boxes as well as ground truth class labels. 

In [ ]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [ ]:
def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(label_map_dict[row['gt_labels']].encode('utf8'))
        classes.append(row['gt_labels'])

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [ ]:
training_data_writer = (tf.python_io.TFRecordWriter(path="/content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/annotations/training_imgs.record"))
testing_data_writer = (tf.python_io.TFRecordWriter(path="/content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/annotations/testing_imgs.record"))

In [ ]:
annotations_df = pd.read_csv("/content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/annotations/grocerydataset/annotations.csv",
                             header=None, names=["file_name","xmin","ymin","xmax","ymax","gt_labels"])

In [ ]:
annotations_df['gt_labels'] = annotations_df['gt_labels'].apply(lambda x: x+1)

In [ ]:
annotations_df

In [ ]:
different_classes = annotations_df['gt_labels'].unique()
print(different_classes)
print(len(different_classes))

# Let's determine all the file names inside the testing folder of our dataset. 

In [ ]:
testing_imgs = os.listdir("/content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/images/ShelfImages/test")

# As our dataset annotations consist of annotations of all the training as well as testing images, therefore it becomes important to seperate out annotations of training as well as testing images. 

In [ ]:
training_data_annotations = list()
testing_data_annotations = list()

for i in range(len(annotations_df)):

  if annotations_df.iloc[i,0] in testing_imgs:
    testing_data_annotations.append(list(annotations_df.iloc[i,:]))
  else:
    training_data_annotations.append(list(annotations_df.iloc[i,:]))

In [ ]:
training_data_annotations_df = pd.DataFrame(data=training_data_annotations,columns=annotations_df.columns)
testing_data_annotations_df = pd.DataFrame(data=testing_data_annotations,columns=annotations_df.columns)

In [ ]:
training_data_annotations_df

In [ ]:
testing_data_annotations_df

# We have created all the functions to create .tfrecord files as well as also create seperate training as well as testing data annotations. Now, let's start the procedure to create .tfrecord files using the above functions. 

# Let's create training as well as testing examples. 

In [ ]:
grouped_training_data = split(df=training_data_annotations_df,group="file_name")
grouped_testing_data = split(df=testing_data_annotations_df,group="file_name")

In [ ]:
training_data_base_path = "/content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/images/ShelfImages/train"
testing_data_base_path = "/content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/images/ShelfImages/test"

In [ ]:
for single_img_group in grouped_training_data:

  tf_example = create_tf_example(group=single_img_group,path=training_data_base_path)
  training_data_writer.write(record=tf_example.SerializeToString())

training_data_writer.close()

In [ ]:
for single_img_group in grouped_testing_data:

  tf_example = create_tf_example(group=single_img_group,path=testing_data_base_path)
  testing_data_writer.write(record=tf_example.SerializeToString())

testing_data_writer.close()

In [ ]:
images_dataset = tf.data.TFRecordDataset("/content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/annotations/training_imgs.record")

In [ ]:
out_img_features = {
        'image/height': tf.io.FixedLenFeature(shape=[],dtype=tf.int64),
        'image/width': tf.io.FixedLenFeature(shape=[],dtype=tf.int64),
        'image/filename': tf.io.FixedLenFeature(shape=[],dtype=tf.string),
        'image/source_id': tf.io.FixedLenFeature(shape=[],dtype=tf.string),
        'image/encoded': tf.io.FixedLenFeature(shape=[],dtype=tf.string),
        'image/format': tf.io.FixedLenFeature(shape=[],dtype=tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(dtype=tf.float32),
        'image/object/class/text': tf.io.FixedLenSequenceFeature(shape=[],dtype=tf.string,allow_missing=True),
        'image/object/class/label': tf.io.VarLenFeature(dtype=tf.int64),
    }

In [ ]:
def parse_image(example_proto):
  return tf.io.parse_single_example(serialized=example_proto,features=out_img_features)

In [ ]:
import IPython.display as display

parsed_image_dataset = images_dataset.map(parse_image)
counter = 0

for image_out_features in parsed_image_dataset:

  if counter == 2:
    break

  print(image_out_features["image/height"])
  print(image_out_features["image/width"])
  print(image_out_features["image/filename"])
  print(image_out_features["image/source_id"])
  print(image_out_features["image/format"])
  print(image_out_features["image/object/bbox/xmin"])
  print(image_out_features["image/object/bbox/xmax"])
  print(image_out_features["image/object/bbox/ymin"])
  print(image_out_features["image/object/bbox/ymax"])
  print(image_out_features["image/object/class/text"])
  print(image_out_features["image/object/class/label"])

  image_out = image_out_features["image/encoded"].numpy()
  display.display(display.Image(data=image_out))

  counter = counter + 1

# Now, we should train our object detection neural network and the neural network which we have chosen from Tensorflow Object Detection Zoo, is: 

# ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8.tar.gz

# Therefore, we are going to download the zip file of this model to /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/pre_trained_models and fine tune it on our dataset. 

In [ ]:
cd /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/pre_trained_models

In [ ]:
! wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8.tar.gz

# Let's now unzip this file at the same location. 

In [ ]:
! tar -xvf /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/pre_trained_models/ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8.tar.gz

# Now, suppose we wish to use some other pre-trained model to train our object detector on the same dataset then we can download other pre-trained model's tarbar files into the "pre_trained_models" directory and untar it here only.  

# Now, let's navigate to the /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/models and create a new directory for our training job for fine tuning our currently downloaded architecture of SSD. 

# Let's first navigate to the above directory and create a seperate directory for our training job inside the above mentione directory. 

In [ ]:
cd /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/models

In [ ]:
! mkdir resnet50_v1_ssd_fpn_1024

# Inside the above created directory, we have to copy the pipeline configuration file (pipeline.config) of the downloaded model inside /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/pre_trained_models/ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8

# to 

# /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/models/resnet50_v1_ssd_fpn_1024

In [ ]:
! cp /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/pre_trained_models/ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8/pipeline.config /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/models/resnet50_v1_ssd_fpn_1024

# Now, let's modify our copied pipeline.config file inside /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection/models/resnet50_v1_ssd_fpn_1024

# And change the following variables in pipeline.config according to your desired configuration:

# (line 3) num_classes: <Total number of classes of different objects in our dataset>

# (line 131) batch_size: <Our desired batch size on which we want to fine tune our pre-trained network on our dataset, if we have high end GPU and good amount of memory then keep it to the default value of 8, otherwise reduce it. We will set it to 4> 

# (line 161) fine_tune_checkpoint: <path of the directory where pre-trained weights of our Neural Network are present in the form of checkpoints and they are going to be fine tuned on our dataset by further training our network on our desired dataset> 

# (line 167) fine_tune_checkpoint_type: <"detection">

# (line 168) use_bfloat16: <Set this variable's value to false if not training on a TPU>

# (line 172) label_map_path: <path of the directory where label_map.pbtxt file is written>

# (line 174) input_path: <path of the directory where TFrecord file of the training data is written>

# (line 178) metrics_set: <"coco_detection_metrics">

# (line 179) use_moving_averages: <"false">

# (line 182) label_map_path: <path of the directory where label_map.pbtxt file is written>

# (line 186) input_path: <path of the directory where TFrecord file of the testing data is written>

# Let's change the directory to root directory of our training configuration

In [ ]:
cd /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection

# Let's copy the file named "model_main_tf2.py" from /content/drive/MyDrive/tf2_od_api/models/research/object_detection/model_main_tf2.py 

# to 

# /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection

# because now we need this script to fine tune our pre-trained neural network on our dataset. 

In [ ]:
! cp /content/drive/MyDrive/tf2_od_api/models/research/object_detection/model_main_tf2.py /content/drive/MyDrive/tf2_od_api/workspace/ciggs_images_detection

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

In [ ]:
! python model_main_tf2.py --model_dir=models/resnet50_v1_ssd_fpn_1024 --pipeline_config_path=models/resnet50_v1_ssd_fpn_1024/pipeline.config

# As we have already trained our model and the trained parameters of our model have been saved in the following directory: 

# /content/drive/MyDrive/tensorflow_od_api/workspace/shelf_objects_detection/models/resnet50_v1_ssd_fpn_1024/train

# Now, we should use these trained parameters to perform inference on our unseen data. 

# Now, in order to do that first we have to export our trained model with trained weights. For that, we have to export our model using a script called exporter_main_v2.py which is located at the following directory: 

# /content/drive/MyDrive/tensorflow_od_api/models/research/object_detection/exporter_main_v2.py

# We have to now copy this script from the above location and paste it into the following directory: 

# /content/drive/MyDrive/tensorflow_od_api/workspace/shelf_objects_detection

# and then run this script.

In [ ]:
!cp /content/drive/MyDrive/tensorflow_od_api/models/research/object_detection/exporter_main_v2.py /content/drive/MyDrive/tensorflow_od_api/workspace/shelf_objects_detection

In [ ]:
cd /content/drive/MyDrive/tensorflow_od_api/workspace/shelf_objects_detection

# Now, since we have copied the script, let's run the script to export our trained model to the following directory: 

# /content/drive/MyDrive/tensorflow_od_api/workspace/shelf_objects_detection/exported_models/first_training_instance

In [ ]:
!python ./exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/resnet50_v1_ssd_fpn_1024/pipeline.config --trained_checkpoint_dir ./models/resnet50_v1_ssd_fpn_1024 --output_directory ./exported_models/first_training_instance

In [ ]:
import random
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from object_detection.utils import ops
from object_detection.utils import visualization_utils as viz
from object_detection.utils.label_map_util import create_category_index_from_labelmap
import glob
from PIL import Image

In [ ]:
def load_image(path):
  image_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(image_data))
  width, height = image.size
  shape = (height, width, 3)
  image = np.array(image.getdata())
  image = image.reshape(shape).astype('uint8')
  return image

In [ ]:
def infer_image(net, image):
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis, ...]
  result = net(input_tensor)
  num_detections = int(result.pop('num_detections'))
  result = {key: value[0, :num_detections].numpy() for key, value in result.items()}
  result['num_detections'] = num_detections
  result['detection_classes'] = result['detection_classes'].astype('int64')
  if 'detection_masks' in result:
    detection_masks_reframed = ops.reframe_box_masks_to_image_masks(result['detection_masks'],result['detection_boxes'],image.shape[0],image.shape[1])
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,tf.uint8)
    result['detection_masks_reframed'] = detection_masks_reframed.numpy()
  return result

In [ ]:
labels_path = "./annotations/label_map.pbtxt"
category_idx = create_category_index_from_labelmap(labels_path,use_display_name=True)
model_path = "./exported_models/first_training_instance/saved_model"
model = tf.saved_model.load(model_path)
test_images = list(glob.glob("./images/ShelfImages/test/*.JPG"))
random.shuffle(test_images)
test_images = test_images[:16]

In [ ]:
for image_path in test_images:
  image = load_image(image_path)
  result = infer_image(model, image)
  masks = result.get('detection_masks_reframed',None)
  viz.visualize_boxes_and_labels_on_image_array(image,result['detection_boxes'],result['detection_classes'],
                                                result['detection_scores'],category_idx,instance_masks=masks,
                                                use_normalized_coordinates=True,line_thickness=5)
  plt.figure(figsize=(24,24))
  plt.imshow(image)
  plt.savefig(f'detections_{image_path.split("/")[-1]}')